<a href="https://colab.research.google.com/github/rlagosb/GastricCancerIncidence/blob/main/1_Trajectories_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup 💾

In [30]:
import pandas as pd
import plotly.graph_objects as go

path_cubo = 'https://raw.githubusercontent.com/rlagosb/GastricCancerIncidence/main/Data/'

# cargar provincias
provincias = (pd.read_parquet(path_cubo + 'CUBO_CANCER_DIGESTIVO.parquet')[['Region','Provincia','Nombre Region','Nombre Provincia']].
              drop_duplicates())


# Cargar egresos-defunciones integradas
egresosDefs = pd.read_parquet(path_cubo + 'EGRESOS_DEFUNCIONES_DESAGREGADOS.parquet')
print(egresosDefs.info())
egresosDefs.head()

<class 'pandas.core.frame.DataFrame'>
Index: 179750 entries, 0 to 193742
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   idPersona    177398 non-null  object 
 1   Sexo         179750 non-null  int64  
 2   Año          179750 non-null  int64  
 3   Edad         179750 non-null  int64  
 4   Provincia    179686 non-null  float64
 5   Categoria    179750 non-null  object 
 6   Trayectoria  179750 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 11.0+ MB
None


,idPersona,Sexo,Año,Edad,Provincia,Categoria,Trayectoria
0,00004077707F5BC1AA117B203BE2833B3F52FAFC855000...,1,2024,83,131.0,C16,Defuncion
1,0000D51AA8378E56D7842D351885F332F7001BCB5FE681...,1,2011,66,101.0,C16,Egreso
2,000279A75E32EA8D99AC4F7E4583179CA9DDDF2681BD91...,1,2011,89,92.0,C16,Egreso
3,000279A75E32EA8D99AC4F7E4583179CA9DDDF2681BD91...,1,2011,89,92.0,C16,Defuncion
5,00043FB7B47538431B765682D5BEF22FACCC0EDF2DB63E...,2,2024,66,135.0,C16,Defuncion


# Casos excluidos
En el periodo objetivo

In [17]:
# Eventos sin id Persona o Provincia
egresosDefs['SinIdPersona'] = (egresosDefs.idPersona.isna())
egresosDefs['SinProvincia'] = (egresosDefs.Provincia==999)

egresosDefs.reset_index().pivot_table(index='Categoria',
                                                     columns=['Trayectoria','SinProvincia','SinIdPersona'],
                                                     values='index',
                                                     aggfunc='count',
                                                     margins=True)

Trayectoria  Defuncion  Egreso                      All
SinProvincia     False   False        True             
SinIdPersona     False   False  True False True        
Categoria                                              
C16              69860  107261  2343   277    9  179750
All              69860  107261  2343   277    9  179750

In [18]:
egresosDefs = egresosDefs[lambda x: (x.PeriodoObjetivo) & ~x.SinProvincia & ~x.SinIdPersona & (x.Provincia!=122)] # 122: Antártica
egresosDefs[egresosDefs.PeriodoObjetivo].reset_index().pivot_table(index='Categoria',
                                                     columns=['Trayectoria','SinProvincia','SinIdPersona'],
                                                     values='index',
                                                     aggfunc='count',
                                                     margins=True)

Trayectoria,Defuncion,Egreso,All
SinProvincia,False,False,
SinIdPersona,False,False,
Categoria,,,
C16,69860,107252,177112
All,69860,107252,177112


# Trayectorias Global

In [19]:
#@title Revisión Trayectorias

def cargar_trayectorias_personas(df):
  # Concatenar eventos CG
  def serie_trayectorias(serie):
    serie = serie.astype(str) + '>'
    x = serie.cumsum()
    return x

  trayectorias = (df.groupby(['idPersona'])['Trayectoria'].
                  apply(serie_trayectorias).str[:-1].value_counts(dropna=False))
  return trayectorias.reset_index()

trayectorias = cargar_trayectorias_personas(egresosDefs.copy())
print(trayectorias.info())
trayectorias.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Trayectoria  383 non-null    object
 1   count        383 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.1+ KB
None


,Trayectoria,count
0,Egreso,62862
1,Defuncion,33223
2,Egreso>Defuncion,23058
3,Egreso>Egreso,20217
4,Egreso>Egreso>Egreso,8359


In [20]:
print(trayectorias['count'].sum(), 'personas con trayectorias de CG,\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Defuncion')]['count'].sum()*100/trayectorias['count'].sum(), '% con defunción\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Egreso')]['count'].sum()*100/trayectorias['count'].sum(), '% con egreso\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Defuncion') & ~x.Trayectoria.str.contains('Egreso')]['count'].sum()*100/trayectorias['count'].sum(), '% con defunción sin egreso')

177112 personas con trayectorias de CG,
 39.46542300916934 % con defunción
 81.24068386105967 % con egreso
 18.75931613894033 % con defunción sin egreso


In [21]:
def diagrama_sankey(trayectorias):

  # Parametrizar transiciones
  for col in ['Sano_Egreso','Sano_Defuncion','Egreso_Vivo','Egreso_Defuncion']:
    trayectorias[col] = False
  egresoInicio = (trayectorias.Trayectoria.str[:6]=='Egreso')
  conDefuncion = (trayectorias.Trayectoria.str.contains('Defuncion'))
  # Sano>Egreso
  trayectorias.loc[egresoInicio,'Sano_Egreso'] = True
  # Sano>Defuncion
  trayectorias.loc[trayectorias.Trayectoria.str[:9]=='Defuncion','Sano_Defuncion'] = True
  # Egreso>Vivo
  trayectorias.loc[egresoInicio & ~conDefuncion,'Egreso_Vivo'] = True
  # Egreso>Defuncion
  trayectorias.loc[egresoInicio & conDefuncion,'Egreso_Defuncion'] = True

  arcos = (trayectorias.melt(id_vars=['Trayectoria','count'],
                           value_vars=['Sano_Egreso','Sano_Defuncion','Egreso_Vivo','Egreso_Defuncion'],
                           var_name='Arco',value_name='Valor')[lambda x: x.Valor].  # dejar sólo transiciones activas
           groupby(['Arco','Valor'])['count'].sum().reset_index())                  # agrupar transiciones
  # separar origen/destino
  arcos[['Source','Target']] = (arcos.Arco.str.replace('Egreso','Hospitalización').str. # cambiar nombre Egreso
                                split('_',expand=True))

  # Create Sankey diagram https://plotly.com/python/sankey-diagram/
  nodes = pd.concat([arcos['Source'], arcos['Target']]).unique()
  nodes = pd.DataFrame({'name': nodes})
  nodes['id'] = nodes.index
  arcos = arcos.merge(nodes.rename(columns={'name':'Source'}), on='Source', how='left')
  arcos = arcos.merge(nodes.rename(columns={'name':'Target'}), on='Target', how='left', suffixes=('_source', '_target'))

  go.Figure(data=[go.Sankey(
      node=dict(pad=15, thickness=20,
          line=dict(color="black", width=0.5),
          label=nodes['name'],),
      link=dict(
          source=arcos['id_source'],
          target=arcos['id_target'],
          value=arcos['count']))]).show()


diagrama_sankey(trayectorias.copy())

# Trayectorias Provincias

In [26]:
def trayectorias_persona(df):
  df = df[df.idPersona.notna()]

  fa = df[df.Trayectoria=='Defuncion'][['idPersona','Año']].rename(columns={'Año':'AñoDefuncion'})
  eg = df[df.Trayectoria=='Egreso'][['idPersona','Año']].rename(columns={'Año':'AñoEgreso'})

  # Obtener año de egreso y hospitalización
  df = (df.groupby(['idPersona']).agg({'Provincia':'first'}).reset_index().
        merge(eg, on='idPersona', how='outer').
        merge(fa, on='idPersona', how='outer').
        groupby(['idPersona','Provincia']).agg({'AñoEgreso':'min','AñoDefuncion':'max'}).reset_index())

  # Métricas sobrevida
  df['Delta'] = df.AñoDefuncion - df.AñoEgreso
  df.loc[df.AñoDefuncion.isna(), 'sobrevida5'] = True
  df.loc[df.Delta.notna(), 'sobrevida5'] = df[df.Delta.notna()].Delta.apply(lambda x: x>=5)

  # get minimum between AñoEgreso and AñoDefuncion
  df['Quinquenio'] = df[['AñoEgreso','AñoDefuncion']].min(axis=1).apply(lambda x: (x//5)*5)

  df['Defuncion'] = df.AñoDefuncion.notna()
  df['Egreso'] = df.AñoEgreso.notna()
  df['DefuncionSinEgreso'] = df.Defuncion & ~df.Egreso

  df = df.groupby(['Provincia','Quinquenio']).agg({'Delta':'mean','sobrevida5':'sum','Defuncion':'sum','Egreso':'sum','DefuncionSinEgreso':'sum','idPersona':'count'}).reset_index()

  return df

trayectoriasProv = trayectorias_persona(egresosDefs.copy())

## Defunciones sin hospitalización

In [25]:
# Defunción sin hospitalización por provincia y quinquenio

print('Porcentaje de defunciones sin hospitalización:\n',
      trayectoriasProv.DefuncionSinEgreso.sum()*100/trayectoriasProv.idPersona.sum())

trayectoriasProv['PorcDefSinEgreso'] = trayectoriasProv.DefuncionSinEgreso*100/trayectoriasProv.idPersona

trayectoriasProv[lambda x: x.Quinquenio>2000].pivot(columns='Quinquenio',index='Provincia',values='PorcDefSinEgreso').\
merge(provincias, on='Provincia').\
set_index(['Region','Nombre Region','Nombre Provincia']).drop(columns='Provincia').round(1)

Porcentaje de defunciones sin hospitalización:
 34.58227057415023


2005.0  \
Region Nombre Region                             Nombre Provincia           
1      Tarapacá                                  Iquique             67.0   
                                                 Tamarugal           77.8   
2      Antofagasta                               Antofagasta         31.9   
                                                 El Loa              29.6   
                                                 Tocopilla           16.0   
3      Atacama                                   Copiapó             34.6   
                                                 Chañaral            23.3   
                                                 Huasco              28.2   
4      Coquimbo                                  Elqui               40.3   
                                                 Choapa              52.2   
                                                 Limarí              39.2   
5      Valparaíso                                Valparaíso          25.4   
                                                 Isla de Pascua      25.0   
                                                 Los Andes           33.3   
                                                 Petorca             37.8   
                                                 Quillota            34.7   
                                                 San Antonio         38.5   
                                                 San Felipe          14.7   
                                                 Marga Marga         37.6   
6      Libertador General Bernardo O'Higgins     Cachapoal           34.7   
                                                 Cardenal Caro       45.8   
                                                 Colchagua           38.1   
7      Maule                                     Talca               44.0   
                                                 Cauquenes           55.1   
                                                 Curicó              40.1   
                                                 Linares             41.4   
8      Biobío                                    Concepción          28.8   
                                                 Arauco              33.4   
                                                 Bíobío              33.1   
9      La Araucanía                              Cautín              37.9   
                                                 Malleco             28.9   
10     Los Lagos                                 Llanquihue          29.0   
                                                 Chiloé              45.1   
                                                 Osorno              37.0   
                                                 Palena              30.6   
11     Aysén del General Carlos Ibáñez del Campo Coyhaique           38.1   
                                                 Aysén               43.9   
                                                 Capitán Prat        60.0   
                                                 General Carrera     38.5   
12     Magallanes y de la Antártica Chilena      Magallanes          30.6   
                                                 Tierra del Fuego    20.0   
                                                 Última Esperanza    52.0   
13     Metropolitana de Santiago                 Santiago            34.4   
                                                 Cordillera          24.4   
                                                 Chacabuco           39.6   
                                                 Maipo               36.7   
                                                 Melipilla           51.5   
                                                 Talagante           53.1   
14     Los Ríos                                  Valdivia            26.4   
                                                 Ranco               24.0   
15     Arica y Parinacota                        Arica         

## Sobrevida (Sup Material 4)


In [28]:
# add colum percentage sobrevida5

# Agregar total nacional
sobre = pd.concat([trayectoriasProv.copy(),
                  trayectoriasProv.groupby(['Quinquenio'])[['sobrevida5','idPersona']].sum().reset_index()])

# Calcular sobrevida 5 años
sobre['PorcSobrevida5'] = (sobre.sobrevida5*100.0/sobre.idPersona).round(1)

print('Sobrevida a 5 años nacional:\n',   sobre[sobre.Provincia.isna()].sobrevida5.sum()/sobre[sobre.Provincia.isna()].idPersona.sum())


Sobrevida a 5 años nacional:
 0.2855967935037218


In [29]:
sobre = sobre.pivot(index='Provincia', columns='Quinquenio', values='PorcSobrevida5').\
merge(provincias, on='Provincia', how='left').\
drop(columns='Provincia').set_index(['Region','Nombre Region','Nombre Provincia'])

for col in sobre.columns:
  sobre[col] = sobre[col].astype(float).round(1)

sobre

2000.0  \
Region Nombre Region                             Nombre Provincia           
NaN    NaN                                       NaN                 26.7   
1.0    Tarapacá                                  Iquique             14.1   
                                                 Tamarugal           33.3   
2.0    Antofagasta                               Antofagasta         23.6   
                                                 El Loa              37.8   
                                                 Tocopilla           15.8   
3.0    Atacama                                   Copiapó             39.3   
                                                 Chañaral            56.2   
                                                 Huasco              13.2   
4.0    Coquimbo                                  Elqui               26.3   
                                                 Choapa              22.2   
                                                 Limarí              20.4   
5.0    Valparaíso                                Valparaíso          25.5   
                                                 Isla de Pascua       NaN   
                                                 Los Andes           32.0   
                                                 Petorca             25.8   
                                                 Quillota            27.1   
                                                 San Antonio         15.5   
                                                 San Felipe          24.7   
                                                 Marga Marga         35.2   
6.0    Libertador General Bernardo O'Higgins     Cachapoal           20.5   
                                                 Cardenal Caro       29.4   
                                                 Colchagua           17.6   
7.0    Maule                                     Talca               16.6   
                                                 Cauquenes           10.3   
                                                 Curicó              16.7   
                                                 Linares             16.9   
8.0    Biobío                                    Concepción          41.3   
                                                 Arauco              38.3   
                                                 Bíobío              23.2   
9.0    La Araucanía                              Cautín              17.4   
                                                 Malleco             21.0   
10.0   Los Lagos                                 Llanquihue          22.8   
                                                 Chiloé              14.8   
                                                 Osorno              26.6   
                                                 Palena              57.1   
11.0   Aysén del General Carlos Ibáñez del Campo Coyhaique           16.7   
                                                 Aysén               41.2   
                                                 Capitán Prat        50.0   
                                                 General Carrera      0.0   
12.0   Magallanes y de la Antártica Chilena      Magallanes          26.6   
                                                 Tierra del Fuego    12.5   
                                                 Última Esperanza    12.5   
13.0   Metropolitana de Santiago                 Santiago            28.8   
                                                 Cordillera          35.5   
                                                 Chacabuco           26.1   
                                                 Maipo               21.9   
                                                 Melipilla           23.9   
                                                 Talagante           18.8   
14.0   Los Ríos                                  Valdivia            27.2   
                                                 Ranco         